# Historical Data Simulation

In [14]:
import pandas as pd
from get_cars import get_cars
from datetime import datetime
import time
from datetime import datetime, timedelta

In [12]:
historical = []

num_intersections = 6
num_days = 50

# One record per hour.
records_per_int = num_days * 24
id = 0

for intersection in range(0, num_intersections):
    for i in range(0, records_per_int):
        timestamp = time.time() - (i * 3600)
        hour = datetime.fromtimestamp(timestamp).hour
        cars = get_cars(hour)
        historical.append([
            id,
            intersection,
            timestamp,
            hour,
            cars
        ])

        id = id+1

data = pd.DataFrame(
    historical,
    columns=[
        'id',
        'intersection',
        'timestamp',
        'hour',
        'cars'
    ],
).set_index('id')

In [29]:
data.to_csv('historical.csv')

In [ ]:
!psql $DATABASE_URL -c "\copy traffic_api_historicaldata (id, intersection,timestamp,hour,cars) from 'historical.csv' delimiter ',' csv;"

In [49]:
granularities = [
    'hourly',
    'daily',
    'weekly',
    'monthly',
    'yearly'
]

def floor_date(ts, granularity):
    '''
    Can pass in one of the granularities, floors the timestamp to the given
    granularity.

    '''

    dt = datetime.fromtimestamp(ts)

    if granularity == 'yearly':
        dt_floor = dt.replace(month=1, day=1, hour=0, minute=0, second=0)
    elif granularity == 'monthly':
        dt_floor = dt.replace(day=1, hour=0, minute=0, second=0)
    elif granularity == 'daily':
        dt_floor = dt.replace(hour=0, minute=0, second=0)

    # Nothing for hourly.

    return time.mktime(dt_floor.timetuple())

In [68]:
granularity = 'daily'
id = 1
start_date = 1490770800.0
end_date = 1490770800.0


results = list(data[data.intersection == id].T.to_dict().values())

10 loops, best of 3: 73.8 ms per loop


In [65]:
%%timeit
response = {
    'meta': {
        'id': id,
        'granularity': granularity,
        'start_date': start_date,
        'end_date': end_date,
        'results': len(results)
    },
    'data': {}
}

for result in results:
    result['cars'] = result['cars'] * 60
    result['timestamp'] = floor_date(result['timestamp'], granularity)

    # Add a new record at this timetamp.
    response['data'][result['timestamp']] = 0

# Aggregate results.
for result in results:
    stored = response['data'][result['timestamp']]
    response['data'][result['timestamp']] = stored + result['cars']

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:22: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: overflow encountered in double_scalars


100 loops, best of 3: 11 ms per loop


In [73]:
import pandas as pd

results = list(data[data.intersection == id].T.to_dict().values())

In [79]:
df = pd.DataFrame(results)
df['datetime'] = pd.to_datetime(df['timestamp'])

In [96]:
datetime.fromtimestamp(1487318400.0)

datetime.datetime(2017, 2, 17, 0, 0)

In [97]:
rsp = {"meta": {"end_date": "1490831240", "results": 1198, "id": "1", "start_date": "149061240", "granularity": "daily"}, "data": {"1490774400.0": 6817.384037734741, "1490688000.0": 7837.599197968131, "1490169600.0": 6869.602688662619, "1486800000.0": 7916.7633464817245, "1489305600.0": 7557.368364147035, "1487491200.0": 7619.887528333634, "1488441600.0": 7157.707346042385, "1489392000.0": 7135.753639640365, "1490256000.0": 7650.326318441402, "1486886400.0": 7400.8782471758705, "1490342400.0": 7155.923884834175, "1487577600.0": 7552.503406087828, "1488528000.0": 7604.702368563361, "1489478400.0": 7964.698775008184, "1490428800.0": 7718.55774584207, "1486972800.0": 7332.785121947016, "1488614400.0": 8022.803416441756, "1487664000.0": 7748.67292434607, "1489564800.0": 7776.709561874931, "1490515200.0": 7525.357520063064, "1488700800.0": 7365.834755812245, "1487059200.0": 7744.604690957611, "1489651200.0": 7699.295722825884, "1487750400.0": 7501.905360762027, "1490601600.0": 7654.575987291482, "1488787200.0": 7970.121687491205, "1488355200.0": 7275.19353973986, "1489737600.0": 7155.845819494222, "1487923200.0": 7256.868132216989, "1487836800.0": 7894.779051315327, "1488873600.0": 6906.432264232839, "1489824000.0": 7753.817851225784, "1489910400.0": 7123.5397405762305, "1488009600.0": 6384.206793762165, "1486540800.0": 6221.197087314915, "1488960000.0": 7075.095779166928, "1487145600.0": 6713.174916893245, "1490083200.0": 7227.502975424442, "1488096000.0": 7325.8585325211125, "1489046400.0": 8120.220704649389, "1486627200.0": 7651.359807550103, "1489996800.0": 7492.526189708389, "1488182400.0": 6506.3946118056665, "1489132800.0": 7474.816949249923, "1487318400.0": 7727.406976647129, "1486713600.0": 7013.314442469741, "1488268800.0": 7362.873616826697, "1487232000.0": 6774.258223350779, "1489219200.0": 8053.172581834075, "1487404800.0": 7590.684209742672}}

In [98]:
rsp['data']

{'data': {'1486540800.0': 6221.197087314915,
  '1486627200.0': 7651.359807550103,
  '1486713600.0': 7013.314442469741,
  '1486800000.0': 7916.7633464817245,
  '1486886400.0': 7400.8782471758705,
  '1486972800.0': 7332.785121947016,
  '1487059200.0': 7744.604690957611,
  '1487145600.0': 6713.174916893245,
  '1487232000.0': 6774.258223350779,
  '1487318400.0': 7727.406976647129,
  '1487404800.0': 7590.684209742672,
  '1487491200.0': 7619.887528333634,
  '1487577600.0': 7552.503406087828,
  '1487664000.0': 7748.67292434607,
  '1487750400.0': 7501.905360762027,
  '1487836800.0': 7894.779051315327,
  '1487923200.0': 7256.868132216989,
  '1488009600.0': 6384.206793762165,
  '1488096000.0': 7325.8585325211125,
  '1488182400.0': 6506.3946118056665,
  '1488268800.0': 7362.873616826697,
  '1488355200.0': 7275.19353973986,
  '1488441600.0': 7157.707346042385,
  '1488528000.0': 7604.702368563361,
  '1488614400.0': 8022.803416441756,
  '1488700800.0': 7365.834755812245,
  '1488787200.0': 7970.12168